# **Códigos úteis para a P2 de Robótica Computacional** 

## OpenCV

### - Filtragem de cores

Criação de limites para a faixa de cores, utilizando HSV.

In [ ]:
g1, g2 = (array([45, 80, 20], dtype=uint8), array([ 65, 255, 255], dtype=uint8))
b1, b2 = (array([105,  80,  20], dtype=uint8), array([125, 255, 255], dtype=uint8))

def mask(hsv, a1, a2):
    return cv2.inRange(hsv, a1, a2)

Ou

In [ ]:
hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)

orange_min = np.array([3,50,50])
orange_max = np.array([20, 255,255])

mask = cv2.inRange(hsv, orange_min, orange_max)

Para chamar a máscara:

In [ ]:
mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

#### Vermelho

In [ ]:
def segmenta_vermelhos(frame):
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    cor_menor = np.array([0, 50, 50])
    cor_maior = np.array([8, 255, 255])
    segmentado_cor = cv2.inRange(frame_hsv, cor_menor, cor_maior)

    cor_menor = np.array([172, 50, 50])
    cor_maior = np.array([180, 255, 255])
    segmentado_cor += cv2.inRange(frame_hsv, cor_menor, cor_maior)
    
    return segmentado_cor

#### Azul

In [ ]:
blue_menor = np.array([105,  50,  50], dtype=uint8)
blue_maior = np.array([125, 255, 255], dtype=uint8)

#### Verde

In [ ]:
green_menor = np.array([45, 50, 50], dtype=uint8) 
green_maior = np.array([65, 255, 255], dtype=uint8)

#### Ciano

In [ ]:
ciano_menor = np.array([80, 50, 50], dtype=uint8)
ciano_maior = np.array([100, 255, 255], dtype=uint8)

#### Magenta

In [ ]:
magenta_menor = np.array([140, 50, 50], dtype=uint8)
magenta_maior = np.array([160, 255, 255], dtype=uint8)

### - Calcular Ponto de Fuga

In [ ]:
def calcula_pf(linha1, linha2):
    x0, y0, x1, y1 = linha1
    m1 = (y1-y0)/(x1-x0)
    h1 = y1 - m1*x1

    x0, y0, x1, y1 = linha2
    m2 = (y1-y0)/(x1-x0)
    h2 = y1 - m2*x1

    x_i = (h2 - h1)/(m1 - m2)
    y_i = m1*x_i + h1

    return int(x_i), int(y_i)

### - Calcular distância

In [ ]:
# Funcao que calcula distancia
def calcula_dist(p1, p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5

### - HoughCircles

Detecta círculos através do HoughCircles.

In [ ]:
bordas = auto_canny(mask)
circles=cv2.HoughCircles(image=bordas,method=cv2.HOUGH_GRADIENT,dp=1.5,minDist=40,param1=200,param2=100,minRadius=5,maxRadius=200)
bordas_rgb = cv2.cvtColor(bordas, cv2.COLOR_GRAY2RGB)
output = bordas_rgb

if circles is not None:        
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        # draw the outer circle
        cv2.circle(output,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle
        cv2.circle(output,(i[0],i[1]),2,(0,0,255),3)

### - Encontra se o símbolo é X ou O

In [ ]:
def encontra_simbolo(contornos) :
    """0 - vazio
       1 - circulo
       2 - x
    """
    if len(contornos) == 0:
        return 0
    else:
        for contorno in contornos:
            if cv2.contourArea(contorno) > 1000:
                if cv2.contourArea(contorno) > 3000:
                    return 1
                else :
                    return 2

## ROS

### - Centro de Massa

Retorna uma tupla (cx, cy) que marca o centro do contorno.

In [ ]:
def center_of_mass(mask):
    M = cv2.moments(mask)
    # Usando a expressão do centróide definida em: https://en.wikipedia.org/wiki/Image_moment
    if M["m00"] == 0:
        M["m00"] = 1
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    return [int(cX), int(cY)]

Para chamar o centro de massa com uma máscara:

In [ ]:
cm = center_of_mass(mask)

Desenha um + (crosshair) centrado no point (tupla (x, y)). 

In [ ]:
def crosshair(img, point, size, color):
    """ color é uma tupla R,G,B uint8 """
    x,y = point
    cv2.line(img,(x - size,y),(x + size,y),color,5)    
    cv2.line(img,(x,y - size),(x, y + size),color,5)

### - Centro da imagem

In [ ]:
cv_image = bridge.compressed_imgmsg_to_cv2(imagem, "bgr8")

In [ ]:
cimg = (int(cv_image.shape[1]/2), int(cv_image.shape[0]/2))

### - Controle do robô

Sempre definir v, w. Quando robô vira à direita, w é negativo.

In [ ]:
w = 0.25
v = 0.25

zero = Twist(Vector3(0,0,0), Vector3(0,0,0))
dire = Twist(Vector3(0,0,0), Vector3(0,0,-w))
esq = Twist(Vector3(0,0,0), Vector3(0,0,w))
frente = Twist(Vector3(v,0,0), Vector3(0,0,0))

Centralizando o robô no frame e no objeto de escolha.

In [ ]:
margem = int(0.15*c_img[0])
# Checar se objeto esta a esq
if c_objeto[x] < c_img[x] - margem:
    pub.publish(esq)
    # Checar se objeto esta a dir
elif c_objeto[x] > c_img[x] + margem:
    pub.publish(dire)
else: 
    # Se centralizado, seguir em frente
    pub.publish(frente)

Trabalhando com estados:

In [ ]:
PROCURANDO, CENTRALIZANDO, AVANCANDO, PARADO = 1, 2, 3, 4

state = PROCURANDO

def printstate(state):
    if state == PROCURANDO:
        print ("PROCURANDO")
    elif state == CENTRALIZANDO:
        print ("CENTRALIZANDO")
    elif state == AVANCANDO:
        print ("AVANCANDO")
    elif state == PARADO:
        print ("PARADO")

dt = 0.05

In [ ]:
while not rospy.is_shutdown():
    printstate(state)
    if state == PROCURANDO:
        velocidade_saida.publish(rot)
        if circle_visible == True:
            state = CENTRALIZANDO
            velocidade_saida.publish(zero)
            rospy.sleep(dt)
    if state == CENTRALIZANDO:          
        print("Delta", circle_delta)
        atuacao = -circle_delta/275.0*0.3
        rot = Twist(Vector3(0,0,0), Vector3(0,0,atuacao))
        velocidade_saida.publish(rot)            
        if abs(circle_delta) < delta_tolerance:
            state = AVANCANDO
            velocidade_saida.publish(zero)              
            rospy.sleep(dt)
    elif state == AVANCANDO:
        velocidade_saida.publish(frente)
        if metro - dist_tolerance < distancia < metro + dist_tolerance:
            state = PARADO
            velocidade_saida.publish(zero)
            rospy.sleep(dt)
    elif state == PARADO:
        velocidade_saida.publish(zero)

### - *Função go_to()*

In [ ]:
def go_to(x1, y1, pub):
    x0 = x # Vai ser atualizado via global e odometria em um thread paralelo
    y0 = y # global e odometria (igual ao acima)
    delta_x = x1 - x0
    delta_y = y1 - y0

    h = math.sqrt(delta_x**2 + delta_y**2) # Distancia ate o destino. Veja 
    # https://web.microsoftstream.com/video/f039d50f-3f6b-4e01-b45c-f2bffd2cbd84

    while h > 0.3:      
        print("Goal ", x1,",",y1)
        # Rotacao
        ang_goal = math.atan2(delta_y,delta_x)  
        ang_atual = rad_z # rad_z muda automaticamente via global e odometria
        dif_ang = ang_goal - ang_atual
        delta_t = abs(dif_ang)/v_ang
        # Twist
        if dif_ang > 0.0:
            vel_rot = Twist(Vector3(0,0,0), Vector3(0,0,v_ang))
        elif dif_ang <=0:
            vel_rot = Twist(Vector3(0,0,0), Vector3(0,0,-v_ang))    
        # publish
        pub.publish(vel_rot)
        # sleep
        rospy.sleep(delta_t)
        zero = Twist(Vector3(0,0,0), Vector3(0,0,0))
        pub.publish(zero)
        rospy.sleep(0.1)
        # Translacao
        delta_t = h/v_lin
        linear = Twist(Vector3(v_lin,0,0), Vector3(0,0,0))
        pub.publish(linear)
        rospy.sleep(delta_t)
        pub.publish(zero)
        rospy.sleep(0.1)  
        x0 = x
        y0 = y
        delta_x = x1 - x0
        delta_y = y1 - y0
        h = math.sqrt(deltScreencast from 15 06 2020 17:39:42

## MobileNet

Essa função recebe uma imagem colorida e devolve o objeto encontrado.

In [ ]:
model = "MobileNetSSD_deploy.caffemodel"
proto = "MobileNetSSD_deploy.prototxt.txt"

video = "animais_caixas.mp4" #Substituir pelo vídeo que quer

def detect(frame):
    image = frame.copy()
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

    # pass the blob through the network and obtain the detections and
    # predictions
    print("[INFO] computing object detections...")
    net.setInput(blob)
    detections = net.forward()

    results = []

    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the
        # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence


        if confidence > CONFIDENCE:
            # extract the index of the class label from the `detections`,
            # then compute the (x, y)-coordinates of the bounding box for
            # the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # display the prediction
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
            print("[INFO] {}".format(label))
            cv2.rectangle(image, (startX, startY), (endX, endY),
                COLORS[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, label, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

            results.append((CLASSES[idx], confidence*100, (startX, startY),(endX, endY) ))

    # show the output image
    return image, results

Recebe um mesmo frame em BGR e como máscara binária, encontra o maior contorno na máscara e desenha o contorno e a caixa envoltória na BGR

In [ ]:
def bounding_box(mask, bgr, color = (0,0,255)):
    contornos, arvore = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    cv2.drawContours(bgr, contornos, -1, [255, 255, 0], 3);
    maior = None
    maior_area = 0
    for c in contornos:
        area = cv2.contourArea(c)
        if area > maior_area:
            maior_area = area
            maior = c
    cv2.drawContours(bgr, [maior], -1, [0, 255, 255], 5);
    
    # Inicializando com valores fora da faixa que vai ser encontrada
    max_x = -1
    min_x = 10000
    max_y = -1
    min_y = 10000
    
    # imprimindo o contorno entendemos sua estrutura
    
    for p in maior:
        if p[0][0] > max_x:
            max_x = p[0][0]
        
        if p[0][0] < min_x:
            min_x = p[0][0]
            
        if p[0][1] < min_y:
            min_y = p[0][1]
        
        if p[0][1] > max_y:
            max_y = p[0][1]
    
    minp = (min_x, min_y)
    maxp = (max_x, max_y)
    
    cv2.rectangle(bgr, minp, maxp, color, 3)
    
    return minp, maxp

Recebe uma tupla de resultados da mobilenet, por exemplo ('cat', 98.16664457321167, (650, 237), (793, 436)) e uma caixa definida por minp e maxp e diz se o animal da tupla está contido na caixa

In [ ]:
def result_in_mask(tupla_mobilenet, minp, maxp):
    t = tupla_mobilenet
    net_min = t[2]
    net_max = t[3]
    
    x = 0
    y = 1
    
    print(net_min, net_max, minp, maxp)
    
    if (net_min[x] > minp[x]) and (net_max[x] < maxp[x]) and (net_min[y] > minp[y]) and (net_max[y] < maxp[y]):
        return True
    else:
        return False    


*Depois da inicialização do programa (if name == main):*

In [ ]:
# Inicializa a aquisição da webcam
cap = cv2.VideoCapture(video)

# cria a rede neural
net = cv2.dnn.readNetFromCaffe(proto, model)

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]   

CONFIDENCE = 0.7
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# Texto na tela

In [ ]:
def text_cv(bgr, pos, text, fontscale=1, thickness=1):
    font = cv2.FONT_HERSHEY_SIMPLEX    
    color=(255,255,255)    
    mensagem = "{}".format(text)
    cv2.putText(bgr, mensagem, pos, font, fontscale, color, thickness, cv2.LINE_AA) 

Chamando a função de texto:

In [ ]:
text_cv(mask_bgr, centro, str((graus/6)))

# BRISK

In [ ]:
# Definir o vídeo/foto a ser trabalhado
video = "video.mp4"

In [ ]:
# Cria o detector BRISK
brisk = cv2.BRISK_create()

# Encontra os pontos únicos (keypoints) nas duas imagems
kp1, des1 = brisk.detectAndCompute(img_original ,None)

# Configura o algoritmo de casamento de features que vê *como* o objeto que deve ser encontrado aparece na imagem
bf = cv2.BFMatcher(cv2.NORM_HAMMING)    

In [ ]:
def find_homography_draw_box(kp1, kp2, img_cena, good):
    
    out = img_cena.copy()
    
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)


    # Tenta achar uma trasformacao composta de rotacao, translacao e escala que situe uma imagem na outra
    # Esta transformação é chamada de homografia 
    # Para saber mais veja 
    # https://docs.opencv.org/3.4/d9/dab/tutorial_homography.html
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()


    
    h,w = img_original.shape
    # Um retângulo com as dimensões da imagem original
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

    # Transforma os pontos do retângulo para onde estao na imagem destino usando a homografia encontrada
    dst = cv2.perspectiveTransform(pts,M)
   
    corners = find_box_corners(dst)
        

    # Desenha um contorno em vermelho ao redor de onde o objeto foi encontrado
    img2b = cv2.polylines(out,[np.int32(dst)],True,(255,255,0),5, cv2.LINE_AA)
    
    return img2b, corners


O exemplo abaixo carrega duas imagens. A `img_original` é o objeto a ser encontrado, e a `img_cena` contém uma cena que contém o objeto.

In [ ]:
# Número mínimo de pontos correspondentes
MIN_MATCH_COUNT = 10

cena_bgr = cv2.imread("arduino_mesa.jpg") # Imagem do cenario
original_bgr = book_bgr

# Versões RGB das imagens, para plot
original_rgb = cv2.cvtColor(original_bgr, cv2.COLOR_BGR2RGB)
cena_rgb = cv2.cvtColor(cena_bgr, cv2.COLOR_BGR2RGB)

In [ ]:
def match(des1, cena_bgr):                

    img_cena = cv2.cvtColor(cena_bgr, cv2.COLOR_BGR2GRAY)
    
    kp2, des2 = brisk.detectAndCompute(img_cena,None)


    # Tenta fazer a melhor comparacao usando o algoritmo
    matches = bf.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    framed = cena_bgr
    bbox=((0,0),(0,0)) # Caixa vazia de retorno padrão
    
    if len(good)>MIN_MATCH_COUNT:
        # Separa os bons matches na origem e no destino
        print("Matches found")    
        framed, bbox = find_homography_draw_box(kp1, kp2, cena_bgr, good)
    else:
        print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    return framed, bbox

*---------------------------------------------------------------------------------------------------------------------------*

### Funções úteis para resoluções de exercícios em BRISK

### - Conta píxeis

Recebe uma máscara binária e 2 pontos e conta quantos pixels são brancos na máscara.

In [ ]:
def count_pixels(mask, ponto1, ponto2, txt_color):
    x1, y1 = ponto1
    x2, y2 = ponto2

    font = cv2.FONT_HERSHEY_COMPLEX_SMALL
    # Selecionando só a região da imagem com o cachorro
    submask = mask[y1:y2,x1:x2]
    # Somando os pixels 255 e dividindo por 255 para saber quantos são
    pixels = np.sum(submask)/255
    # O resto é só plot
    rgb_mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    cv2.rectangle(rgb_mask, ponto1, ponto2, (255,0,0), 3)
    cv2.putText(rgb_mask, "%s:%d"%(txt_color, pixels), (int((x1+x2)/2), int((y1+y2)/2)), font, 1, (0,255,0),1,cv2.LINE_AA)
    return pixels, rgb_mask
    #plt.imshow(submask, cmap="Greys_r", vmin=0, vmax=255)
    #plt.title("(%d , %d) (%d,%d)"%(x1, y1, x2, y2))
    #plt.show()
    

tolerancia = 0.25

### - Transfroma representação de pontos em matriz

Transforma uma representação de 4 pontos em uma matriz usada pelo cv2 perspectiveTransform em uma lista de tuplas (x,y).        

In [ ]:
def make_list(pts):
    # Remova os comentários para entender como os pontos são armazenados na cv2
    # print("Points:\n", pts) # Para salientar como encontrar os pontos
    # print("dst.shape: ", pts.shape)
    l = []
    for p in pts:
        x = p[0][0]
        y = p[0][1]
        l.append((x,y))
    return l
    

### - Bounding box

Encontra os cantos da bounding box.

In [ ]:
def find_box_corners(pts):        
    """ Versao mais didatica """
    l = make_list(pts)
    print("l: ", l)
    x_coords = [p[0] for p in l]
    y_coords = [p[1] for p in l]
    x_min = int(min(x_coords))
    x_max = int(max(x_coords))
    y_min = int(min(y_coords))
    y_max = int(max(y_coords))       
    return ((x_min,y_min),(x_max,y_max))

def find_box_corners2(pts):
    return ((min(pts[:,:,0]), min(pts[:,:,1])), (max(pts[:,:,0]), max(pts[:,:,1])))


### - Processa frame (adaptável para cada código, segue o exemplo do ex2 da p2_2020)

Recebe um frame BGR, procura seu padrão e calcula a área do padrão encontrado na imagem. Depois separa o canal vermelho contando a quantidade e proporção de pixels vermelhos. Se a proporção for maior que o que foi desejado, printa a mesnagem na tela.

In [ ]:
def processa(frame):
    """Recebe um frame BGR"""
    # Procura o padrao
    red_match, corners = match(des1, frame)
    # Calcula a área do padrão encontrado na imagem
    area_found = area(corners[0], corners[1])
    
    proporcao = -1 # inicializando com um valor imnpossível
    
    # Separa o canal vermelho
    red = segmenta_vermelhos(frame)
    
    # Conta a quantidade e a proporcao de vermelhos
    qtd_vermelhos, saida_count = count_pixels(red,corners[0],corners[1], (0,0,255))  


    cv2.imshow("debug", saida_count)
    
    if area_found>0:
        proporcao = qtd_vermelhos/area_found    
    
    if area_found > 0 and proporcao > tolerancia:
        cv2.rectangle(red_match, corners[0],corners[1], (0,0,255), 10)
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL        
        cv2.putText(red_match, "Encontrado", (corners[0][0], corners[1][1]), font, 3, (0,0,255),2,cv2.LINE_AA)
        print("returning red_match")
        return red_match
    else:
        return frame

def area(pt1, pt2):
    return abs((pt1[0] - pt2[0])*(pt1[1] - pt2[1]))